In [1]:
import pandas as pd
import os
import re
from evaluate import is_int

def fix_accuracies(filepath):
    
    for filename in os.listdir(filepath):
        if filename.endswith('.csv'):
            
            df = pd.read_csv(os.path.join(filepath, filename))

            accuracies = []
            for idx, row in df.iterrows():
                model_output = row["model_cot"]
                true_answer = row["true_answer"]

                marker = r"Therefore, the answer \(arabic numerals\) is "
                matches = list(re.finditer(marker, model_output))

                if matches:
                    # Step 2: Get the last match
                    last_match = matches[-1]
                    
                    # Step 3: Get the text after the last match
                    remaining_text = model_output[last_match.end():]
                    
                    # Step 4: Search for the first number after the last match
                    number_match = re.search(r'\d+', remaining_text)
                    if number_match:
                        pred = int(number_match.group())
                    elif len(matches) > 1:
                        second_last_match = matches[-2]
                        remaining_text = model_output[second_last_match.end():]
                        number_match = re.search(r'\d+', remaining_text)
                        if number_match:
                            pred = int(number_match.group())
                        else:
                            pred = -1
                    else:
                        pred = -1
                else:
                    pred = -1

                df.loc[idx, "model_answer"] = pred
                accuracies.append(int(int(pred) == true_answer) if is_int(pred) else 0)

            df["accuracy"] = sum(accuracies)/len(accuracies)

            df.to_csv(os.path.join(filepath, filename), index=False) 


/Users/andreasopedal/Documents/research/proof-search/mwp-proof-search/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 10-15 14:36:42 [__init__.py:239] Automatically detected platform cpu.
